<a href="https://colab.research.google.com/github/Worachet-Ch/Project/blob/main/XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project2022' #เข้า drive
data_test = pd.read_csv(os.path.join(path,'test15.csv'))
data_test.drop('Unnamed: 0',1,inplace = True)

Mounted at /content/drive


<ipython-input-2-22381b44fb0a>:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_test.drop('Unnamed: 0',1,inplace = True)


In [ ]:
from xgboost import XGBRegressor
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score

In [ ]:
data_test

,Sex,AgeSYear,telomere length (kb),Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,39,7.57,44.4,148,20.3,91,66,81,18.0,...,0.9,5.0,170,50,65,95,12,20,35,5.2
1,2,25,10.30,50.5,158,20.2,122,70,86,20.5,...,0.5,3.9,165,40,68,89,13,28,32,5.3
2,2,42,9.71,57.1,158,22.9,107,65,81,22.8,...,0.7,3.9,183,55,60,112,18,26,60,5.5
3,1,26,8.36,59.7,170,20.7,122,68,75,27.6,...,1.2,5.6,186,87,45,124,22,27,43,7.8
4,2,71,5.02,40.9,150,18.2,128,62,83,17.6,...,0.7,3.6,196,111,58,115,15,29,52,4.4
5,2,36,8.18,62.5,160,24.4,86,55,86,23.0,...,0.7,4.1,194,65,58,123,27,31,45,5.5
6,2,46,6.45,55.2,163,20.8,138,85,71,22.0,...,0.7,5.1,219,85,56,146,10,23,53,5.7
7,2,52,8.15,53.9,156,22.1,139,84,80,20.7,...,1.1,6.9,184,75,72,97,17,34,64,7.1
8,2,30,9.04,51.1,158,20.5,108,66,74,20.4,...,0.5,5.4,158,56,34,112,12,20,61,3.9
9,1,32,7.46,70.3,177,22.4,110,75,76,31.0,...,1.0,5.5,212,75,58,139,35,26,57,6.3


In [ ]:
def data(Train, Test):
  y_train =Train[['telomere length (kb) ']].values.copy() #n = 407
  X_train = Train.drop('telomere length (kb) ',axis = 1).values.copy() # n =407
  X_test = Test.drop('telomere length (kb) ',axis = 1).values.copy()
  y_test = Test[['telomere length (kb) ']].values.copy()
  model = XGBRegressor()
  param = {'learning_rate': [0.1,0.05],
        'max_depth': [3, 6,10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.3,0.5,1],
        'colsample_bytree': [ 0.3, 0.5, 0.7, 1],
        'n_estimators' : [100,300, 500,1000],
        'seed':[216],'alpha':[0, 0.05, 0.5]}
  search = GridSearchCV(model, param, scoring='neg_mean_absolute_error', n_jobs=1, cv=5, refit=True)
  result = search.fit(X_train, y_train)
  best_model = result.best_estimator_
  best_model.fit(X_train,y_train) 
  scores_mae = cross_val_score(best_model , X_train, y_train, scoring='neg_mean_absolute_error',cv=5 )
  y_pred = best_model.predict(X_test)
  mae  = metrics.mean_absolute_error(y_test, y_pred)
  print('Best model:',best_model)
  print(f'score:{scores_mae}')
  print(f'Mean score:{scores_mae.mean()}')
  print('MAE Test:',mae)
  print('R2 Value Test:',metrics.r2_score(y_test, y_pred))
  print('MSE Test:',metrics.mean_squared_error(y_test, y_pred))

In [ ]:
def cross(data):
  y_train =data[['telomere length (kb) ']].values.copy() #n = 407
  X_train = data.drop('telomere length (kb) ',axis = 1).values.copy() # n =407
  X_test = data_test.drop('telomere length (kb) ',axis = 1).values.copy()
  y_test = data_test[['telomere length (kb) ']].values.copy()
  learning_rate = [0.1,0.05] 
  max_depth = [3,6,10]
  min_child_weight = [1,3,5]
  subsample = [1]
  colsample_bytree = [1]
  n_estimators = [100]
  alpha = [0,0.5,0.05]
  for a in  learning_rate:
    for b in max_depth:
      for c in min_child_weight :
        for d in subsample:
          for e in colsample_bytree:
            for f in n_estimators :
              for g in alpha:
                model = XGBRegressor(learning_rate = a,max_depth= b ,min_child_weight = c,subsample= d,colsample_bytree = e,n_estimators= f,seed = 216,alpha = g)
                model.fit(X_train,y_train) 
                scores_mae = cross_val_score(model , X_train, y_train, scoring='neg_mean_absolute_error',cv=5 )
                y_pred = model.predict(X_test)
                mae  = metrics.mean_absolute_error(y_test, y_pred)
                print('model:', model) 
                print(f'score:{scores_mae}')
                print(f'Mean score:{scores_mae.mean()}')
                print('MAE Test:',mae)
                print('R2 Value Test:',metrics.r2_score(y_test, y_pred))
                print('MSE Test:',metrics.mean_squared_error(y_test, y_pred))
                print('-----------------------------')

# พร้อมกันทุกตัวแปร

In [ ]:
df_once = pd.read_csv(os.path.join(path,'df_once.csv'))
df_once.drop('Unnamed: 0',1,inplace = True)

<ipython-input-6-788f31b5dfa3>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_once.drop('Unnamed: 0',1,inplace = True)


In [ ]:
df_once

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c,telomere length (kb)
0,2,58,47.200,156.00,19.400,112.00,64.00,70.00,19.709,13.578,...,4.8,180,97,62,98,25,32.0,40.0,5.800,7.30
1,2,57,47.400,153.00,20.200,114.00,75.00,69.00,19.300,12.500,...,5.0,176,51,72,94,16,16.0,62.0,5.100,6.36
2,2,24,53.200,157.00,21.600,103.00,69.00,78.00,21.300,14.600,...,5.2,159,45,52,98,13,17.0,50.0,5.400,9.71
3,2,62,54.400,165.00,20.000,95.00,75.00,83.00,21.600,15.200,...,4.1,230,94,61,151,18,31.0,76.0,5.600,6.17
4,2,86,49.000,154.00,20.700,129.00,79.00,83.00,19.967,13.705,...,5.4,183,86,39,127,12,22.0,82.0,6.200,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2,25,47.000,165.00,17.300,104.00,63.00,89.00,20.443,12.801,...,4.8,232,36,96,128,18,12.0,46.0,5.113,8.98
403,2,31,72.200,168.00,25.600,123.00,89.00,69.00,25.313,18.577,...,6.1,187,49,51,127,21,24.0,71.0,5.426,8.23
404,2,32,62.315,155.91,25.276,112.53,72.11,77.96,22.803,24.156,...,5.1,222,58,79,132,17,10.0,34.0,5.357,8.79
405,1,29,60.132,168.49,21.862,116.63,75.98,81.99,27.959,11.444,...,6.8,140,92,48,73,31,75.0,69.0,6.033,8.93


In [ ]:
data(df_once,data_test)

Best model: XGBRegressor(alpha=0.5, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.62141273 -0.63886942 -0.62248531 -0.71272635 -0.90800362]
Mean score:-0.7006994878527129
MAE Test: 0.6692672424316408
R2 Value Test: 0.6830645034136452
MSE Test: 0.6499408905916242


In [ ]:
cross(df_once)

model: XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.74856043 -0.66438509 -0.6413324  -0.75757776 -0.88113141]
Mean score:-0.7385974175626346
MAE Test: 0.682328479766846
R2 Value Test: 0.6758101634843252
MSE Test: 0.6648173945020406
-----------------------------
model: XGBRegressor(alpha=0.5, base_score=None, boos

# ทีละตัวแปร จากค่าว่างมากไปน้อย

In [ ]:
df_decreasing = pd.read_csv(os.path.join(path,'df_decreasing.csv'))
df_decreasing.drop('Unnamed: 0',1,inplace = True)

<ipython-input-11-9b6b50a13b92>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_decreasing.drop('Unnamed: 0',1,inplace = True)


In [ ]:
df_decreasing

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c,telomere length (kb)
0,2,58,47.200000,156.000000,19.400000,112.000,64.00,70.00,20.208,16.356,...,4.8,180,97,62,98,25,32.0,40.0,5.800000,7.30
1,2,57,47.400000,153.000000,20.200000,114.000,75.00,69.00,19.300,12.500,...,5.0,176,51,72,94,16,16.0,62.0,5.100000,6.36
2,2,24,53.200000,157.000000,21.600000,103.000,69.00,78.00,21.300,14.600,...,5.2,159,45,52,98,13,17.0,50.0,5.400000,9.71
3,2,62,54.400000,165.000000,20.000000,95.000,75.00,83.00,21.600,15.200,...,4.1,230,94,61,151,18,31.0,76.0,5.600000,6.17
4,2,86,49.000000,154.000000,20.700000,129.000,79.00,83.00,19.709,14.545,...,5.4,183,86,39,127,12,22.0,82.0,6.200000,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2,25,47.000000,165.000000,17.300000,104.000,63.00,89.00,22.658,15.876,...,4.8,232,36,96,128,18,12.0,46.0,5.586667,8.98
403,2,31,72.200000,168.000000,25.600000,123.000,89.00,69.00,22.956,17.269,...,6.1,187,49,51,127,21,24.0,71.0,5.233333,8.23
404,2,32,57.733763,161.138144,22.066778,110.860,72.25,74.74,21.605,14.816,...,5.1,222,58,79,132,17,10.0,34.0,5.593333,8.79
405,1,29,59.594922,166.059308,21.357616,113.945,75.82,78.13,30.868,11.115,...,6.8,140,92,48,73,31,75.0,69.0,5.420000,8.93


In [ ]:
data(df_decreasing,data_test)

Best model: XGBRegressor(alpha=0.05, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.69339353 -0.72950194 -0.58666433 -0.53327982 -0.907221  ]
Mean score:-0.690012123898762
MAE Test: 0.7749959678649904
R2 Value Test: 0.5716453699477293
MSE Test: 0.878428553266712


In [ ]:
cross(df_decreasing)

model: XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.7011952  -0.66575832 -0.60251264 -0.72218104 -0.91349762]
Mean score:-0.7210289621801127
MAE Test: 0.6476175041198734
R2 Value Test: 0.6976744325311179
MSE Test: 0.6199802505107065
-----------------------------
model: XGBRegressor(alpha=0.5, base_score=None, boo

# ทีละตัวแปร จากค่าว่างน้อยไปมาก

In [ ]:
df_increasing = pd.read_csv(os.path.join(path,'df_increasing.csv'))
df_increasing.drop('Unnamed: 0.1',1,inplace = True)
df_increasing.drop('Unnamed: 0',1,inplace = True)

<ipython-input-14-acef49fd91fe>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_increasing.drop('Unnamed: 0.1',1,inplace = True)
<ipython-input-14-acef49fd91fe>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_increasing.drop('Unnamed: 0',1,inplace = True)


In [ ]:
df_increasing

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c,telomere length (kb)
0,2,58,47.200000,156.000000,19.400000,112.00,64.00,70.00,19.588,13.636,...,4.8,180,97,62,98,25,32.0,40.0,5.800000,7.30
1,2,57,47.400000,153.000000,20.200000,114.00,75.00,69.00,19.300,12.500,...,5.0,176,51,72,94,16,16.0,62.0,5.100000,6.36
2,2,24,53.200000,157.000000,21.600000,103.00,69.00,78.00,21.300,14.600,...,5.2,159,45,52,98,13,17.0,50.0,5.400000,9.71
3,2,62,54.400000,165.000000,20.000000,95.00,75.00,83.00,21.600,15.200,...,4.1,230,94,61,151,18,31.0,76.0,5.600000,6.17
4,2,86,49.000000,154.000000,20.700000,129.00,79.00,83.00,19.973,14.361,...,5.4,183,86,39,127,12,22.0,82.0,6.200000,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2,25,47.000000,165.000000,17.300000,104.00,63.00,89.00,20.286,14.208,...,4.8,232,36,96,128,18,12.0,46.0,5.480000,8.98
403,2,31,72.200000,168.000000,25.600000,123.00,89.00,69.00,25.127,19.207,...,6.1,187,49,51,127,21,24.0,71.0,5.233333,8.23
404,2,32,58.229814,161.337130,22.316020,113.03,70.77,74.35,23.161,21.327,...,5.1,222,58,79,132,17,10.0,34.0,5.573333,8.79
405,1,29,58.651179,165.311575,21.143898,118.45,76.77,86.54,27.366,18.731,...,6.8,140,92,48,73,31,75.0,69.0,5.160000,8.93


In [ ]:
data(df_increasing,data_test)

Best model: XGBRegressor(alpha=0.5, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.73789195 -0.73502076 -0.62462285 -0.53693084 -0.90954882]
Mean score:-0.7088030420525611
MAE Test: 0.7696738573710123
R2 Value Test: 0.5625959326778454
MSE Test: 0.896986270473812


In [ ]:
cross(df_increasing)

model: XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.74290257 -0.66023021 -0.67224782 -0.74413305 -0.9127497 ]
Mean score:-0.7464526709154835
MAE Test: 0.6531846097310388
R2 Value Test: 0.6985742400930315
MSE Test: 0.6181350115442599
-----------------------------
model: XGBRegressor(alpha=0.5, base_score=None, boo

# ข้อมูลสุ่มเติมชุด 1

In [ ]:
df_Random_1 = pd.read_csv(os.path.join(path,'df_Random_1.csv'))
df_Random_1.drop('Unnamed: 0',1,inplace = True)

<ipython-input-17-fdc99b7ebf69>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_Random_1.drop('Unnamed: 0',1,inplace = True)


In [ ]:
df_Random_1

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c,telomere length (kb)
0,2,58,47.200000,156.000000,19.400000,112.000,64.00,70.00,19.556,15.752,...,4.8,180,97,62,98,25,32.0,40.0,5.800000,7.30
1,2,57,47.400000,153.000000,20.200000,114.000,75.00,69.00,19.300,12.500,...,5.0,176,51,72,94,16,16.0,62.0,5.100000,6.36
2,2,24,53.200000,157.000000,21.600000,103.000,69.00,78.00,21.300,14.600,...,5.2,159,45,52,98,13,17.0,50.0,5.400000,9.71
3,2,62,54.400000,165.000000,20.000000,95.000,75.00,83.00,21.600,15.200,...,4.1,230,94,61,151,18,31.0,76.0,5.600000,6.17
4,2,86,49.000000,154.000000,20.700000,129.000,79.00,83.00,19.717,16.245,...,5.4,183,86,39,127,12,22.0,82.0,6.200000,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2,25,47.000000,165.000000,17.300000,104.000,63.00,89.00,20.263,13.955,...,4.8,232,36,96,128,18,12.0,46.0,5.546667,8.98
403,2,31,72.200000,168.000000,25.600000,123.000,89.00,69.00,25.383,22.256,...,6.1,187,49,51,127,21,24.0,71.0,5.246667,8.23
404,2,32,57.396724,160.346047,22.173580,112.690,70.83,72.79,22.659,20.312,...,5.1,222,58,79,132,17,10.0,34.0,5.353333,8.79
405,1,29,66.214342,171.956726,22.271332,111.605,75.38,83.33,29.393,13.284,...,6.8,140,92,48,73,31,75.0,69.0,5.533333,8.93


In [ ]:
data(df_Random_1,data_test)

In [ ]:
cross(df_Random_1)

model: XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.74351226 -0.63623027 -0.60636457 -0.73359292 -0.90457045]
Mean score:-0.7248540943689068
MAE Test: 0.6587626368204756
R2 Value Test: 0.6841605300107103
MSE Test: 0.6476932644649165
-----------------------------
model: XGBRegressor(alpha=0.5, base_score=None, boo

# ข้อมูลสุ่มเติมชุด 2

In [ ]:
df_Random_2 = pd.read_csv(os.path.join(path,'df_Random_2.csv'))
df_Random_2.drop('Unnamed: 0',1,inplace = True)

<ipython-input-20-8d12976e41d7>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_Random_2.drop('Unnamed: 0',1,inplace = True)


In [ ]:
df_Random_2

,Sex,AgeSYear,Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,Fat Mass,...,Uric Acid,Cholesterol,Triglyceride,HDL-C,LDL,AST,ALT,Alkaline Phos,HbA1c,telomere length (kb)
0,2,58,47.200000,156.000000,19.400000,112.000,64.000000,70.00,19.882,12.598,...,4.8,180,97,62,98,25,32.0,40.0,5.800000,7.30
1,2,57,47.400000,153.000000,20.200000,114.000,75.000000,69.00,19.300,12.500,...,5.0,176,51,72,94,16,16.0,62.0,5.100000,6.36
2,2,24,53.200000,157.000000,21.600000,103.000,69.000000,78.00,21.300,14.600,...,5.2,159,45,52,98,13,17.0,50.0,5.400000,9.71
3,2,62,54.400000,165.000000,20.000000,95.000,75.000000,83.00,21.600,15.200,...,4.1,230,94,61,151,18,31.0,76.0,5.600000,6.17
4,2,86,49.000000,154.000000,20.700000,129.000,79.000000,83.00,19.823,13.827,...,5.4,183,86,39,127,12,22.0,82.0,6.200000,4.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,2,25,47.000000,165.000000,17.300000,104.000,63.000000,89.00,20.844,10.078,...,4.8,232,36,96,128,18,12.0,46.0,5.480000,8.98
403,2,31,72.200000,168.000000,25.600000,123.000,89.000000,69.00,25.324,22.487,...,6.1,187,49,51,127,21,24.0,71.0,5.446667,8.23
404,2,32,53.787249,160.038949,20.905205,107.070,71.439606,74.56,21.663,14.119,...,5.1,222,58,79,132,17,10.0,34.0,5.513333,8.79
405,1,29,67.601595,171.279267,22.892886,115.685,69.085582,88.73,29.540,12.944,...,6.8,140,92,48,73,31,75.0,69.0,5.166667,8.93


In [ ]:
data(df_Random_2,data_test)

In [ ]:
cross(df_Random_2)

model: XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.76350449 -0.6856745  -0.62520368 -0.74345039 -0.91674694]
Mean score:-0.7469160003535757
MAE Test: 0.7478777936299645
R2 Value Test: 0.6422497569110963
MSE Test: 0.7336398545033873
-----------------------------
model: XGBRegressor(alpha=0.5, base_score=None, boo

# ข้อมูลควบคุม

In [ ]:
df_control = pd.read_csv(os.path.join(path,'control.csv'))
df_control.drop('Unnamed: 0',1,inplace = True)

<ipython-input-23-690f813aeb21>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_control.drop('Unnamed: 0',1,inplace = True)


In [ ]:
df_control

,Sex,AgeSYear,telomere length (kb),Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57,6.36,47.4,153,20.2,114,75,69,19.3,...,1.0,5.0,176,51,72,94,16,16,62,5.1
1,2,24,9.71,53.2,157,21.6,103,69,78,21.3,...,0.8,5.2,159,45,52,98,13,17,50,5.4
2,2,62,6.17,54.4,165,20.0,95,75,83,21.6,...,0.7,4.1,230,94,61,151,18,31,76,5.6
3,2,35,7.23,56.3,161,21.7,95,57,72,22.5,...,0.6,5.0,198,47,68,120,16,20,35,5.1
4,1,46,6.52,72.3,170,25.0,133,77,107,29.2,...,0.9,6.9,194,63,60,122,15,19,69,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,1,60,5.81,72.0,170,24.9,131,74,69,30.5,...,0.9,6.9,182,179,39,107,36,31,79,6.5
134,1,58,5.05,62.0,168,22.0,103,76,72,27.8,...,1.0,6.8,191,62,47,132,16,33,42,6.8
135,2,42,9.48,55.5,165,20.4,102,52,77,22.9,...,0.8,4.7,212,68,63,135,11,16,33,4.8
136,2,61,6.06,61.8,159,24.4,117,60,72,23.4,...,0.8,5.1,235,66,56,166,15,23,44,5.4


In [ ]:
df_control

In [ ]:
cross(df_control)

model: XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, ...)
score:[-0.57309997 -0.81379715 -0.84167031 -0.65047882 -0.8050341 ]
Mean score:-0.7368160696130582
MAE Test: 0.81755552037557
R2 Value Test: 0.5052011587461811
MSE Test: 1.0146859629545717
-----------------------------
model: XGBRegressor(alpha=0.5, base_score=None, boost